In [1]:
import pandas as pd
import requests
import json
from datetime import date
import numpy as np

API_KEY = "KEY"

In [2]:
REP_NUM = 7
COMPOUND_LAYOUT = 1
TX_PLATE_BARCODE = "12345"

CELL_TYPE = "52b2636a-e77c-4574-b6f2-78ad283c489e" # -> hard-coded
CULTURE_STATE = "Active" # -> hard-coded
PASSAGE = 8
PARENT_CULTURE = "CUL053"

NUM_CELLS_SEEDED = 400_000
VOL = 0.17

TX_PROTOCOL_ID = "PR003-V1"
PREP_BY = "Yang Gao"

In [3]:
treatment_info = pd.read_csv("../../data/Treatment_Layout_for_testing.csv")
treatment_info

,Compound ID,Compound Aliquot ID,Dose,Dose Unit,Condition,Plate Well
0,TAL501,CMS101,20.0,uM,Test compound,D6
1,TAL502,CMS102,20.0,uM,Test compound,E3
2,TAL503,CMS103,20.0,uM,Test compound,G12
3,TAL504,CMS104,20.0,uM,Test compound,E5
4,TAL505,CMS105,20.0,uM,Test compound,A6
...,...,...,...,...,...,...
87,dBET6,CMS201,50.0,nM,Positive control,B6
88,NaN,NaN,NaN,NaN,DMSO,C10
89,NaN,NaN,NaN,NaN,DMSO,B4
90,NaN,NaN,NaN,NaN,DMSO,A8


In [5]:
def find_row_by_id(manager, id):
    session = requests.Session()
    api_call = 'https://api.scispot.io/v2/labsheets/find-row-by-id'
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "id": id
    }
    ret = session.post(api_call, json=payload)
    # print(ret.text)
    return json.loads(ret.text)

In [6]:
# extract info
def define_compound_inputs(df_row, item):
    if df_row["Condition"].lower() == "dmso":
        name = "THP1_" + "DMSO" + "_rep" + str(REP_NUM)
        tx_stock_id = ""
        tx_dose = ""
        tx_dose_unit = ""
        
    else:
        name = "THP1_" + df_row["Compound ID"] + "_rep" + str(REP_NUM)
        tx_stock_id = find_row_by_id("Compound Stock", df_row["Compound Aliquot ID"])['rows'][0][0]# df_row["Compound Aliquot ID"]
        tx_dose = df_row["Dose"]
        tx_dose_unit = df_row["Dose Unit"]

    if item:
        name = name + item

    tx_type = df_row["Condition"]
    
    return cell_culture_rows(name, tx_type, tx_stock_id, tx_dose, tx_dose_unit)

In [7]:
def cell_culture_rows(name, tx_type, tx_stock_id, tx_dose, tx_dose_unit):
    rows = [
    "AUTOMATIC", # Registry ID - don't do anything with   Registry ID,
    CELL_TYPE, # Cell Type - HARD CODED - THP1       Cell Type,
    name, # Name (temporary field)                   Name,
    CULTURE_STATE, # Culture State                   Culture State,
    "", # Stock Type                                 Stock Type,
    "", # Mycoplasma Test                            Mycoplasma Test,
    PASSAGE, # Passage                               Passage,
    "", # Supplier                                   Supplier,
    "", # Supplier Batch ID                          Supplier Batch ID,
    "", # Received Date                              Received Date,
    PARENT_CULTURE, # Parent Culture                 Parent Culture,
    NUM_CELLS_SEEDED, # Number of Cells Seeded       Number of Cells Seeded,
    VOL, # Volume (mL)                               Volume (mL),
    tx_type, # Treatment Type                        Treatment Type,
    tx_stock_id, # Treatment Compound Stock ID       Treatment Compound Stock ID,
    tx_dose, # Treatment Dose                        Treatment Dose,
    tx_dose_unit, # Treatment Dose Unit              Treatment Dose Unit,
    "", # Child Cultures                             Child Cultures,
    "", # Freezing Protocol ID                       Freezing Protocol ID,
    "", # Culture Protocol ID                        Culture Protocol ID,
    "", # Treatment Protcol ID           Treatment Protocol ID,
    PREP_BY, # Prepared By                           Prepared By,
    str(date.today()), # Preparation Date            Preparation Date,
    "", # Record Creator                             Record Creator,
    ""# Storage Location                            Storage Location
    ]
    return rows
#
#

In [8]:
def add_rows(manager, rows):
    session = requests.Session()
    api_call = 'https://api.scispot.io/v2/labsheets/add-rows'
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "rows": [
            rows
        ]
    }
    ret = session.post(api_call, json=payload)
    print(ret.text)
    return json.loads(ret.text)

In [9]:
manifest_list = []

In [10]:
dmso = 'a'
dbet = 'a'
for index, row in treatment_info.iterrows():
    item = None
    dmso = dmso
    dbet = dbet
    if "dmso" in row["Condition"].lower():
        item = dmso
        dmso = chr(ord(dmso) + 1)
    elif "control" in row["Condition"].lower():
        item = dbet
        dbet = chr(ord(dbet) + 1)
    row_input = define_compound_inputs(row, item)
    print(row_input)
    manifest_list.append({"well": row["Plate Well"], "name": row_input[2]}) # uuid: res[0]["uuid"]     id col: row_input[2]    registry id: temp["rows"][0][1]
    res = add_rows("Cell Culture", row_input)
    print()

['AUTOMATIC', '52b2636a-e77c-4574-b6f2-78ad283c489e', 'THP1_TAL501_rep7', 'Active', '', '', 8, '', '', '', 'CUL053', 400000, 0.17, 'Test compound', 'cab6c280-59e6-45a4-be59-e3255a0ae872', 20.0, 'uM', '', '', '', '', 'Yang Gao', '2023-07-31', '', '']
[{"row":"1","success":"true","uuid":"9f35d7a1-8538-4a7f-8b97-f7013585fa62"}]

['AUTOMATIC', '52b2636a-e77c-4574-b6f2-78ad283c489e', 'THP1_TAL502_rep7', 'Active', '', '', 8, '', '', '', 'CUL053', 400000, 0.17, 'Test compound', 'b5f4c926-7532-4561-9ead-ba3014cbecf6', 20.0, 'uM', '', '', '', '', 'Yang Gao', '2023-07-31', '', '']
[{"row":"1","success":"true","uuid":"3c958c1d-428e-47c3-976c-1185e00538fd"}]

['AUTOMATIC', '52b2636a-e77c-4574-b6f2-78ad283c489e', 'THP1_TAL503_rep7', 'Active', '', '', 8, '', '', '', 'CUL053', 400000, 0.17, 'Test compound', '51ffb568-9427-4c6a-8cfe-8f2873afb2a2', 20.0, 'uM', '', '', '', '', 'Yang Gao', '2023-07-31', '', '']
[{"row":"1","success":"true","uuid":"2334f42e-18b2-4b20-9aea-4bb9074a4cc9"}]

['AUTOMATIC', '5

In [11]:
print(manifest_list)

[{'well': 'D6', 'name': 'THP1_TAL501_rep7'}, {'well': 'E3', 'name': 'THP1_TAL502_rep7'}, {'well': 'G12', 'name': 'THP1_TAL503_rep7'}, {'well': 'E5', 'name': 'THP1_TAL504_rep7'}, {'well': 'A6', 'name': 'THP1_TAL505_rep7'}, {'well': 'A11', 'name': 'THP1_TAL506_rep7'}, {'well': 'F9', 'name': 'THP1_TAL507_rep7'}, {'well': 'H2', 'name': 'THP1_TAL508_rep7'}, {'well': 'E2', 'name': 'THP1_TAL509_rep7'}, {'well': 'E8', 'name': 'THP1_TAL510_rep7'}, {'well': 'G10', 'name': 'THP1_TAL511_rep7'}, {'well': 'G7', 'name': 'THP1_TAL512_rep7'}, {'well': 'H5', 'name': 'THP1_TAL513_rep7'}, {'well': 'H8', 'name': 'THP1_TAL514_rep7'}, {'well': 'A10', 'name': 'THP1_TAL515_rep7'}, {'well': 'C11', 'name': 'THP1_TAL516_rep7'}, {'well': 'C7', 'name': 'THP1_TAL517_rep7'}, {'well': 'E7', 'name': 'THP1_TAL518_rep7'}, {'well': 'C5', 'name': 'THP1_TAL519_rep7'}, {'well': 'G3', 'name': 'THP1_TAL520_rep7'}, {'well': 'G9', 'name': 'THP1_TAL521_rep7'}, {'well': 'C4', 'name': 'THP1_TAL522_rep7'}, {'well': 'F12', 'name': 'T

In [12]:
def make_new_plate(manager, well_info, manifest_name):
    url = "https://api.scispot.io/tryingtofixcors/manifest/create"
    payload = {
        "apiKey": API_KEY,
        "name": manifest_name,
        "template": "96-well plate",
        "plates": [
            {
                "template": "96-well plate",
                "wells": 96,
                "labsheets": [
                    {
                        "labsheet": manager,
                        "items": well_info
                    }
                ]
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
# make_new_plate()

In [13]:
# ## make manifest
manifest_name = TX_PLATE_BARCODE + "_Set" + str(COMPOUND_LAYOUT) + "_Rep" + str(REP_NUM) + "_Treatment"
print(manifest_name)

12345_Set1_Rep7_Treatment


In [14]:
make_new_plate("Cell Culture", manifest_list, manifest_name)

{"uuid":"9694e5ac-9b16-43e9-9796-19c78f2b57cf","success":true}
